# Notebook 1: Retrieval – “Encuentra en tus Apuntes”

Convertir texto en vectores e indexarlo para luego buscar por similitud semántica.


## 1. Instalar Dependencias

In [1]:
!pip install weaviate-client sentence-transformers PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.1 MB/s eta 0:00:00


## 2. Crear documentos de ejemplo (simulación de apuntes)

In [24]:
notes = [
    {
        "title": "Ciclo de vida del software",
        "subject": "Fundamentos de Ingeniería del Software",
        "year": 2023,
        "text": "El ciclo de vida del software incluye etapas como análisis de requisitos, diseño, implementación, pruebas y mantenimiento."
    },
    {
        "title": "Componentes de Hadoop",
        "subject": "Infraestructuras de Big Data",
        "year": 2024,
        "text": "Hadoop está compuesto por HDFS para almacenamiento, YARN para gestión de recursos y MapReduce para procesamiento."
    },
    {
        "title": "Modelo ágil SCRUM",
        "subject": "Fundamentos de Ingeniería del Software",
        "year": 2024,
        "text": "SCRUM es un marco de trabajo ágil que facilita la gestión de proyectos mediante iteraciones cortas y roles bien definidos."
    },
    {
        "title": "Introducción a Spark",
        "subject": "Infraestructuras de Big Data",
        "year": 2023,
        "text": "Apache Spark es una herramienta para el procesamiento en memoria de grandes volúmenes de datos, ideal para análisis rápidos."
    },
    {
        "title": "Control de versiones con Git",
        "subject": "Fundamentos de Ingeniería del Software",
        "year": 2024,
        "text": "Git permite gestionar versiones del código fuente de forma distribuida, facilitando el trabajo colaborativo entre desarrolladores."
    }
]


## 3. Conectarse a Weaviate y definir el esquema

In [25]:
import weaviate
from weaviate.classes.config import Configure, DataType, VectorDistances

client = weaviate.connect_to_local(
    host="rag-weaviate",
    port=8080,
    grpc_port=50051,
)

if client.collections.exists("Note"):
    client.collections.delete("Note")

client.collections.create(
    name="Note",
    properties=[
        {"name": "title", "data_type": DataType.TEXT},
        {"name": "text", "data_type": DataType.TEXT},
        {"name": "subject", "data_type": DataType.TEXT},
        {"name": "year", "data_type": DataType.INT},
    ],
    vectorizer_config=Configure.Vectorizer.none(),  # No usamos vectorizador interno
    vector_index_config=Configure.VectorIndex.hnsw(
        distance_metric=VectorDistances.COSINE  
    )
)


print(client.is_ready())  # Should print: `True`

True


## 4. Insertar Documentos

In [26]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
collection = client.collections.get("Note")

for note in notes:
    vector = model.encode(note["text"])
    collection.data.insert(note, vector=vector)
    print(note["title"],": indexed")


Ciclo de vida del software : indexed
Componentes de Hadoop : indexed
Modelo ágil SCRUM : indexed
Introducción a Spark : indexed
Control de versiones con Git : indexed


/tmp/ipykernel_89/4092201714.py:4: ResourceWarning: unclosed <socket.socket fd=73, family=2, type=1, proto=6, laddr=('172.18.0.4', 53190), raddr=('172.18.0.3', 8080)>
  collection = client.collections.get("Note")


## 5. Consultas

### a) Semánticas

In [27]:
query = "procesamiento distribuido"
vector = model.encode(query)

results = collection.query.near_vector(
    vector, 
    limit=5,
    return_metadata=["distance"]
)

for r in results.objects:
    print(f"[{r.metadata.distance:.3f}] {r.properties['title']} - {r.properties['subject']}")


[0.585] Control de versiones con Git - Fundamentos de Ingeniería del Software
[0.591] Ciclo de vida del software - Fundamentos de Ingeniería del Software
[0.647] Modelo ágil SCRUM - Fundamentos de Ingeniería del Software
[0.653] Introducción a Spark - Infraestructuras de Big Data
[0.701] Componentes de Hadoop - Infraestructuras de Big Data


### b) Meta-Información

In [28]:
from weaviate.classes.query import Filter

results = collection.query.fetch_objects(
    filters=Filter.by_property("subject").equal("Fundamentos de Ingeniería del Software"),
    limit=10
)

for r in results.objects:
    print(r.properties["title"], "-", r.properties["subject"])


Ciclo de vida del software - Fundamentos de Ingeniería del Software
Modelo ágil SCRUM - Fundamentos de Ingeniería del Software
Control de versiones con Git - Fundamentos de Ingeniería del Software


In [29]:
from weaviate.classes.query import Filter

results = collection.query.fetch_objects(
    filters=Filter.by_property("year").less_than(2024),
    limit=10
)

for r in results.objects:
    print(r.properties["title"], "-", r.properties["year"])


Ciclo de vida del software - 2023
Introducción a Spark - 2023


In [32]:
from weaviate.classes.query import Filter

results = collection.query.fetch_objects(
    filters=Filter.all_of([
        Filter.by_property("subject").equal("Fundamentos de Ingeniería del Software"),
        Filter.by_property("year").less_than(2024)
    ]),
    limit=10
)

for r in results.objects:
    print(r.properties["title"], "-", r.properties["year"])


Ciclo de vida del software - 2023


### c) Híbrida

`alpha` define cuánto peso damos a cada tipo de búsqueda en una consulta híbrida:

- `alpha = 0.0`: solo palabras clave (BM25)
- `alpha = 1.0`: solo semántica (vectores)
- `alpha = 0.5`: combinación equilibrada

In [35]:
query = "procesamiento distribuido"
vector = model.encode(query)

results = collection.query.hybrid(
    query=query,
    vector=vector,
    alpha=0.25,
    limit=10,
    return_metadata=["score"]
)

for r in results.objects:
    print(f"[{r.metadata.score:.3f}] {r.properties['title']} - {r.properties['subject']}")


[0.750] Componentes de Hadoop - Infraestructuras de Big Data
[0.250] Control de versiones con Git - Fundamentos de Ingeniería del Software
[0.238] Ciclo de vida del software - Fundamentos de Ingeniería del Software
[0.115] Modelo ágil SCRUM - Fundamentos de Ingeniería del Software
[0.103] Introducción a Spark - Infraestructuras de Big Data


# Anexo 1. Leer todos los PDFs de una carpeta

In [36]:
import os
from PyPDF2 import PdfReader

carpeta_pdfs = "./pdfs"  

for archivo in os.listdir(carpeta_pdfs):
    if archivo.endswith(".pdf"):
        ruta_pdf = os.path.join(carpeta_pdfs, archivo)
        print(f"\n Leyendo: {archivo}")

        # Abrir el PDF y extraer el texto
        lector = PdfReader(ruta_pdf)
        texto = ""
        for pagina in lector.pages:
            texto += pagina.extract_text()

        print(texto[:500], "...")  # Mostramos solo los primeros 500 caracteres


/opt/conda/lib/python3.11/site-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(



 Leyendo: 2312.10997v5.pdf
1
Retrieval-Augmented Generation for Large
Language Models: A Survey
Yunfan Gaoa, Yun Xiongb, Xinyu Gaob, Kangxiang Jiab, Jinliu Panb, Yuxi Bic, Yi Daia, Jiawei Suna, Meng
Wangc, and Haofen Wanga,c
aShanghai Research Institute for Intelligent Autonomous Systems, Tongji University
bShanghai Key Laboratory of Data Science, School of Computer Science, Fudan University
cCollege of Design and Innovation, Tongji University
Abstract —Large Language Models (LLMs) showcase impres-
sive capabilities but e ...


## 1.1 Obtener metadatos de un PDF 

In [41]:
from PyPDF2 import PdfReader

ruta_pdf = "./pdfs/2312.10997v5.pdf"
lector = PdfReader(ruta_pdf)

# ✅ Metadatos (diccionario con la info)
meta = lector.metadata

print("Título:", meta.title)
print("Autor:", meta.author)
print("Asunto:", meta.subject)
print("Creador:", meta.creator)
print("Productor:", meta.producer)
print("Fecha de creación:", meta.creation_date)
print("Número de páginas:", len(lector.pages))


Título: None
Autor: 
Asunto: 
Creador: LaTeX with hyperref
Productor: pdfTeX-1.40.25
Fecha de creación: 2024-03-28 00:54:45+00:00
Número de páginas: 21


# Anexo 2. Dividir un documento en "chunks" (trozos)

In [37]:
def dividir_en_chunks(texto, tamano=500):
    return [texto[i:i+tamano] for i in range(0, len(texto), tamano)]

# Ejemplo con un texto cualquiera
texto = "Este es un documento muy largo ..." * 50  # Imagina que es el texto del PDF
chunks = dividir_en_chunks(texto, tamano=500)

print(f"Se han creado {len(chunks)} chunks")
print("Primer chunk:\n", chunks[0])


Se han creado 4 chunks
Primer chunk:
 Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy


## 2.1 División avanzada en chunks mediante LangChain

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Tamaño de cada chunk
    chunk_overlap=50     # Superposición entre chunks (para no perder contexto)
)

chunks = text_splitter.split_text(texto)
print(f"Se han creado {len(chunks)} chunks")
print("Primer chunk:\n", chunks[0])


Se han creado 4 chunks
Primer chunk:
 Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy largo ...Este es un documento muy
